In [10]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
recipes = pd.read_csv('Recepten_kiezen_gescheiden.csv')
data_recipes = pd.read_csv('data_recipes.csv')
ingr_schoon=pd.read_csv('tabellen/data_ingr_clean.csv') 
fav_recepten = recipes[:80].copy()
fav_cal = fav_recepten[fav_recepten['calories']>= 600]
rec_cal = data_recipes[(data_recipes['calories']>= 550) & (data_recipes['calories']<= 925)].copy()
rec_cal.to_csv('rec_cal.csv', index=False)

In [ ]:
class Recepten: 
    
    def __init__(self, data_ingr, schoongemaakt=False): 
        # Initialisatie van de klasse met het pad naar de data en of de data schoon is 
        self.data_ingr = data_ingr
        self.schoongemaakt = schoongemaakt
        
    def laad_ingr(self):
        # Methode die de ingrediëntentabel inlaadt
        self.tabel = pd.read_csv(self.data_ingr)
        
    def aanbeveling(self, fav_recepten):
        # Alle favoriete ingredienten worden gezocht, dmv set worden de duplicates weggehaald
        self.fav_ingr = list(set(self.tabel['ingredient'].loc[self.tabel['recipe'].isin(fav_recepten)]))
        # Initialisatie van de CountVectorizer die text data convergeert naar getallen
        count_vectorizer = CountVectorizer()
        # Convergeert fav_ingr naar een matrix van tokens
        self.fav_matrix = count_vectorizer.fit_transform(self.fav_ingr)
        # Initialiseer een lege lijst om alle recepten op te slaan
        titel_lijst = []
        # Itereer door alle recepten, maar filter de favoriete eruit en voorkom ook dubbele receptnamen
        [titel_lijst.append(recept) for recept in self.tabel['recipe'] if recept not in fav_recepten and recept not in titel_lijst]
        # Initialiseer een lege lijst om de gelijkenisscore in te bewaren tussen de fav recepten en alle andere
        gelijkenis = []
        # Bereken de cosinusgelijkenis tussen de fav recepten en alle ander
        for recept in titel_lijst:
            # Kies de ingredientelijst van het huidige recept
            recept_lijst = list(self.tabel['ingredient'].loc[self.tabel['recipe']==recept])
            # Convergeer de ingredientenlijst naar een matrix van token obv de fav_matrix
            rec_matrix = count_vectorizer.transform(recept_lijst)
            # Bereken de cosinusgelijkenis tussen het huidige recept en de favoriete ingredienten, 
            # dmv van een vector bestaand uit enen, evengroot als het aantal ingredienten in het huidige recept
            cosinus = cosine_similarity(np.ones(np.size(rec_matrix.toarray())).reshape(1,-1), rec_matrix.reshape(1,-1))
            # Voeg de gelijkenis score en het recept toe aan de lijst
            gelijkenis.append([cosinus[0][0],recept])
        
        # Sorteer de lijst aan gelijkenissen op aflopende volgorde
        gelijkenis.sort(reverse=True)
        # Selecteer de top 10 aanbevolen recepten op basis van de cosinusgelijkenis
        self.aanbeveling = [titel[1] for titel in gelijkenis][:10]
        
        return self.aanbeveling
        
recepten_klasse = Recepten('tabellen/data_ingr_clean.csv',schoongemaakt=True)
recepten_klasse.laad_ingr()

rec = recipes_wel[recipes_wel['calories']>= 600]
fav_recepten = rec['title'].tolist()
aanbevolen = recepten_klasse.aanbeveling(fav_recepten)
aanbevolen